# Setup

In [1]:
import tweepy
import json

api_key = 'CYNk3gteB84Yc74GHuB5HMQHy'
api_secret = 'hlscuCisPhrfL1bD2aA5igLp95pbQW4jNxaWXtPgSAjDpSkIS1'
access_token = '764225096642736128-MWzQKUROrQe9Ha7fHQoGI0pWwnJrtbC'
access_secret = '9gPX4hpP7bMPvaUxI87FMC8MdcMju89OO6nWVCiRyerJY'

# Getting tweets from a specific user

In [2]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)
tweets = api.user_timeline(screen_name = 'SHAQ', count=3)

In [3]:
tweet = tweets[0]._json

#json.dumps takes a json object and dumps it into a string for viewing
#sort_keys sorts in alphabetical order, since sets don't have an order like a list
output = json.dumps(tweet, sort_keys=True, indent=4)

#the actual 'tweet' itself is in the attribute 'text'
#want to keep note of the language attribute in mass tweet downloads (ie flu means something else in indonesian)
#'user' --> 'created_at' is when user created account, not when tweet was created
#ie) tweet['created_at'] != tweet['user']['created_at']
print(output)

{
    "contributors": null,
    "coordinates": null,
    "created_at": "Wed Sep 04 15:30:21 +0000 2019",
    "entities": {
        "hashtags": [],
        "symbols": [],
        "urls": [
            {
                "display_url": "twitter.com/i/web/status/1\u2026",
                "expanded_url": "https://twitter.com/i/web/status/1169271489104887810",
                "indices": [
                    116,
                    139
                ],
                "url": "https://t.co/Ggcbilpmya"
            }
        ],
        "user_mentions": [
            {
                "id": 32576623,
                "id_str": "32576623",
                "indices": [
                    66,
                    81
                ],
                "name": "The General",
                "screen_name": "TheGeneralAuto"
            }
        ]
    },
    "favorite_count": 372,
    "favorited": false,
    "geo": null,
    "id": 1169271489104887810,
    "id_str": "1169271489104887810",
    "in_repl

# Sampling tweets in real time (streaming API)

In [4]:
# CLASS #1 NOTES

# REST vs. Streaming APIs:
# Streaming : not asking for data, twitter is giving it to you (tweets come in real time)
# REST : what we did above with request and response

class MyStreamListener(tweepy.StreamListener):
    #this function is activated anytime a new tweet is pushed into the stream
    #data is what Twitter is giving us
    def on_data(self, data):
        #json.loads takes a string and puts it into a json object (b/c twitter gives us data in a string)
        data = json.loads(data)
        if'text' in data:
            #better idea to write this to a file than to print it
            print(data['text'])
            
myListener = MyStreamListener()
myStream = tweepy.Stream(auth=api.auth, listener=myListener)

#you can give track a list of the keywords that you want (there's a limit on how many, but like 300-400)
# myStream.filter(track=['python'])
#myStream.sample() --> this will give you random sampling of tweets as they come


F♥E♥A♥T♥U♥R♥E♥D♥ C♥O♥U♥R♥S♥E♥

#Python Bible: Everything You Need to Program in Python

Build 11 Projects and go fr… https://t.co/aEyKgqPhGp


KeyboardInterrupt: 

In [10]:
# CLASS #2 NOTES

class MyStreamListener(tweepy.StreamListener):
    def __init__(self):
        self.tweet_count = 0
        self.word_counts = {}
        
    def update_counts(self, text):
        #.split() takes a string into a list of strings (splits by spaces automatically)
        words = text.split()
        
        for word in words:
            if word[0] != '#': continue
                
            if word not in self.word_counts:
                self.word_counts[word] = 0
            self.word_counts[word] += 1
    
    def on_data(self, data):
        self.tweet_count += 1
        
        data = json.loads(data)
        if'text' in data:
            text = data['text']
            self.update_counts(text)
            
            if self.tweet_count % 100 == 0:
                #.get in this case is the values of the dictionary, so this will sort the words by frequency
                #not by the alphabetical order of the words (the original key)
                #we put it in reverse which puts it from largest to smallest, its the other way by default
                for word in sorted(self.word_counts, key=self.word_counts.get, reverse=True)[:5]:
                    print(word, self.word_counts[word])
                print('=============')
            
myListener = MyStreamListener()
myStream = tweepy.Stream(auth=api.auth, listener=myListener)

myStream.sample()

#토묘_자트 1
#ParaRecordar 1
#SharpiePresident… 1
#USJ 1
#USJファン 1
#토묘_자트 1
#ParaRecordar 1
#SharpiePresident… 1
#USJ 1
#USJファン 1
#HarryPotter 2
#CursedChild 2
#Tutuştular 2
#토묘_자트 1
#ParaRecordar 1


KeyboardInterrupt: 

### Ideas for filtering words to see more interesting results
- whitelist --> looking for specific words
- blacklist --> excluding specific words (like stopwords: ie) and, the, then, etc...)


### Seeing anomolies
ie) for hurricane dorian:
you could have a time series that shoes the normal tweets about a word based on time, and if there is a spike on a given day or week, that would be an indicator that it is trending
- but be careful because if somethign has only been tweeted once, and then the next week 2 more people tweet about it, it has only been tweeted 3 times now, even though that's a 3x jump, it isn't necessarily 'trending'
- also like when Steve Jobs died, he didn't trend on twitter, because he was already so popular on twitter

Solutions:
- can remove stop words OR you can adjust values of stop words so they don't get weighted as much
- can do statistical methods (next week) for noticing trending stuff


### Advice on Hmwk

don't open and close a file every time a tweet is posted (ie don't do it on_data)
make a different function in the class where open_file is a variable that exists in the class outside of the on_data
function so it happens once at the beginning of the class instantiation not every tweet
because opening and closing files takes a lot of time